# USA Produce Price Prediction

### Load needed files

In [319]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import TimeSeriesSplit

In [320]:
train_df = pd.read_csv('stocked_tomato2.csv')
train_df.head()

,Date,Price,Season,FMC,CVGW,DE,Rating
0,2002-05-01,89.6,0.5,1.132779,1.059828,1.054862,60
1,2002-06-01,115.9,1.0,1.167839,0.985112,1.090969,80
2,2002-07-01,133.1,1.0,0.979278,1.054355,1.128463,80
3,2002-08-01,102.7,1.0,0.985175,0.995449,1.131597,80
4,2002-09-01,87.2,1.0,0.969056,1.039476,1.170757,80


In [321]:
test_df = pd.read_csv('stocked_tomato2_test.csv')
test_df.head()

,Date,Price,Season,FMC,CVGW,DE,Rating
0,2017-01-01,194.3,0.0,9.014649,11.939167,7.871172,40
1,2017-02-01,215.0,0.0,9.343085,13.069696,7.922163,40
2,2017-03-01,210.9,0.0,11.277569,13.569718,7.940816,40
3,2017-04-01,256.7,0.0,11.398496,14.408596,8.454859,40
4,2017-05-01,273.4,0.5,11.683143,14.449393,9.077213,60


In [322]:
predictors = ['Price','Season','FMC','CVGW','DE']
trainset = pd.DataFrame.drop(train_df, labels='Date', axis = 1)
target = "Price"
trainset.head()

,Price,Season,FMC,CVGW,DE,Rating
0,89.6,0.5,1.132779,1.059828,1.054862,60
1,115.9,1.0,1.167839,0.985112,1.090969,80
2,133.1,1.0,0.979278,1.054355,1.128463,80
3,102.7,1.0,0.985175,0.995449,1.131597,80
4,87.2,1.0,0.969056,1.039476,1.170757,80


### we must normalize our data for neural networks to perform optimally

In [323]:
from sklearn.preprocessing import MinMaxScaler

In [324]:
scaler = MinMaxScaler(feature_range=(0, 1))
# Scale both the training inputs and outputs
scaled_train = scaler.fit_transform(trainset)
#scaled_test = scaler.transform(test_df)

/home/gavelkind/anaconda3/envs/exML/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [325]:
# Print out the adjustment that the scaler applied to the total_earnings column of data
print("Note: median values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[4], scaler.min_[4]))


Note: median values were scaled by multiplying by 0.1516653537 and adding -0.159986


In [326]:
multiplied_by = scaler.scale_[4]
added = scaler.min_[4]

In [327]:
print(type(scaled_train))

<class 'numpy.ndarray'>


In [328]:
scaled_train_df = pd.DataFrame(scaled_train, columns=trainset.columns.values)

In [329]:
scaled_train_df.head()

,Price,Season,FMC,CVGW,DE,Rating
0,0.005993,0.5,0.053489,0.006096,0.000000,0.5
1,0.071660,1.0,0.056716,0.000067,0.005476,1.0
2,0.114607,1.0,0.039364,0.005654,0.011163,1.0
3,0.038702,1.0,0.039906,0.000901,0.011638,1.0
4,0.000000,1.0,0.038423,0.004454,0.017577,1.0


# Let't build our tf.keras model

In [330]:
model = tf.keras.Sequential()

In [331]:
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [332]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [333]:
X = scaled_train_df.values
Y = scaled_train_df[['Price'][:]].values
#old: X = scaled_train_df.drop(target, axis=1).values
#Y = scaled_train_df[[target]].values

In [334]:
# Train the model
model.fit(
    X[10:],
    Y[10:],
    epochs=50,
    shuffle=True,
    verbose=2
)

Epoch 1/50
 - 0s - loss: 0.0621
Epoch 2/50
 - 0s - loss: 0.0338
Epoch 3/50
 - 0s - loss: 0.0185
Epoch 4/50
 - 0s - loss: 0.0148
Epoch 5/50
 - 0s - loss: 0.0087
Epoch 6/50
 - 0s - loss: 0.0049
Epoch 7/50
 - 0s - loss: 0.0026
Epoch 8/50
 - 0s - loss: 0.0015
Epoch 9/50
 - 0s - loss: 0.0011
Epoch 10/50
 - 0s - loss: 7.8707e-04
Epoch 11/50
 - 0s - loss: 4.4615e-04
Epoch 12/50
 - 0s - loss: 2.7401e-04
Epoch 13/50
 - 0s - loss: 1.9649e-04
Epoch 14/50
 - 0s - loss: 1.4062e-04
Epoch 15/50
 - 0s - loss: 2.3860e-04
Epoch 16/50
 - 0s - loss: 1.4374e-04
Epoch 17/50
 - 0s - loss: 9.7706e-05
Epoch 18/50
 - 0s - loss: 6.8408e-05
Epoch 19/50
 - 0s - loss: 6.2812e-05
Epoch 20/50
 - 0s - loss: 5.7808e-05
Epoch 21/50
 - 0s - loss: 5.0101e-05
Epoch 22/50
 - 0s - loss: 5.2582e-05
Epoch 23/50
 - 0s - loss: 4.4178e-05
Epoch 24/50
 - 0s - loss: 4.4768e-05
Epoch 25/50
 - 0s - loss: 4.0394e-05
Epoch 26/50
 - 0s - loss: 3.6919e-05
Epoch 27/50
 - 0s - loss: 3.6100e-05
Epoch 28/50
 - 0s - loss: 3.2501e-05
Epoch 29/

In [335]:
test_error_rate = model.evaluate(X[:10], Y[:10], verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

The mean squared error (MSE) for the test data set is: 3.2102550903800875e-05


In [336]:
prediction = model.predict(X[:1])

In [337]:
y_0 = prediction[0][0]
print('Prediction with scaling - {}'.format(y_0))
y_0 -= added
y_0 /= multiplied_by
print("Price Prediction  - ${}".format(y_0))

Prediction with scaling - 0.0029892059974372387
Price Prediction  - $1.0745707891492378


In [338]:
Y_0 = Y[0]
print('Ground truth with scaling - {}'.format(Y_0))
Y_0 -= added
Y_0 /= multiplied_by

print('Ground Truth Price - ${}'.format(Y_0))

Ground truth with scaling - [0.00599251]
Ground Truth Price - $[1.09437296]


# tf.estimator
## with numpy

In [339]:
# crim = train_df['crim'].values
# zn = train_df['zn'].values
# indus = train_df['indus'].values
# chas = train_df['chas'].values
# nox = train_df['nox'].values
# rm = train_df['rm'].values
# age = train_df['age'].values
# dis = train_df['dis'].values
# rad = train_df['rad'].values
# tax = train_df['tax'].values
# ptratio = train_df['ptratio'].values
# black = train_df['black'].values
# lstat = train_df['lstat'].values
# medv = train_df['medv'].values
price = train_df['Price']
season = train_df['Season']
fmc = train_df['FMC']
cvgw = train_df['CVGW']
de = train_df['DE']

In [340]:
x_dict = {
    'Price': price,
    'Season': season,
    'FMC': fmc,
    'CVGW': cvgw,
    'DE': de,
}

In [341]:
def np_training_input_fn(x, y):
  return tf.estimator.inputs.numpy_input_fn(
      x= x,
      y= y,
      batch_size= 32,
      num_epochs= 5, # this way you can leave out steps from training
      shuffle= True,
      queue_capacity= 5000
  )

In [342]:
feature_cols = [tf.feature_column.numeric_column(k) for k in x_dict.keys()]

In [343]:
lin_model = tf.estimator.LinearRegressor(feature_columns=feature_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpbxennyj4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd4e459afd0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [344]:
lin_model.train(np_training_input_fn(x_dict, price), steps=10)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpbxennyj4/model.ckpt.
INFO:tensorflow:loss = 1627944.1, step = 1
INFO:tensorflow:Saving checkpoints for 10 into /tmp/tmpbxennyj4/model.ckpt.
INFO:tensorflow:Loss for final step: 143487.61.


# tf.estimator
## with pandas

In [345]:
def pd_input_fn(df, y_label):
  return tf.estimator.inputs.pandas_input_fn(
      x=df,
      y=df[y_label],
      batch_size = 32,
      num_epochs = 5,
      shuffle = True,
      queue_capacity = 1000,
      num_threads = 1
  )

In [346]:
lin_model = tf.estimator.LinearRegressor(feature_columns=feature_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp6_oh6hpw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd4e459a6d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [347]:
lin_model.train(pd_input_fn(train_df, 'Price'), steps=10)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp6_oh6hpw/model.ckpt.
INFO:tensorflow:loss = 1200210.0, step = 1
INFO:tensorflow:Saving checkpoints for 10 into /tmp/tmp6_oh6hpw/model.ckpt.
INFO:tensorflow:Loss for final step: 66355.19.


In [348]:
x_dict.keys()

dict_keys(['Price', 'Season', 'FMC', 'CVGW', 'DE'])